In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import load_model

from tqdm import tqdm

In [3]:
# Set the path to the parent directory containing train and validation folders
train_dir = "/Users/damodargupta/Desktop/breast cancer prediction ann/Breast Cancer Patients MRI's/train"
test_dir = "/Users/damodargupta/Desktop/breast cancer prediction ann/Breast Cancer Patients MRI's/validation"


In [4]:
# Set the image dimensions
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)

In [5]:
# Set the batch size and number of epochs
batch_size = 32
epochs = 10

In [6]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(rescale=1./255,  # normalize pixel values between 0 and 1
                                   shear_range=0.2,  # random geometric transformations
                                   zoom_range=0.2,
                                   horizontal_flip=True)

In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

Found 1400 images belonging to 2 classes.


In [9]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(img_width, img_height),
                                                  batch_size=batch_size,
                                                  class_mode='binary')

Found 80 images belonging to 2 classes.


In [10]:
# Build the ANN model
model = Sequential()
model.add(Flatten(input_shape=input_shape))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [11]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
# Train the model
print(train_generator.samples)
print(batch_size)
steps_per_epoch = train_generator.samples // batch_size
print(steps_per_epoch)
validation_steps = test_generator.samples // batch_size

1400
32
43


In [13]:
# Define a custom callback to stop training at a certain validation accuracy threshold
class StopTrainingCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_accuracy') >= 0.75:
            print("\nValidation accuracy reached 80.0% or higher. Stopping training.")
            self.model.stop_training = True

In [228]:
# Train the model ann 
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=epochs,
          validation_data=test_generator,
          validation_steps=test_generator.samples // batch_size,
          callbacks=[StopTrainingCallback()])

Epoch 1/10
43/43 [==============================] - 6s 140ms/step - loss: 0.5159 - accuracy: 0.7500 - val_loss: 0.7824 - val_accuracy: 0.5312
Epoch 2/10
43/43 [==============================] - 5s 127ms/step - loss: 0.5438 - accuracy: 0.7310 - val_loss: 0.5855 - val_accuracy: 0.6719
Epoch 3/10
43/43 [==============================] - 6s 129ms/step - loss: 0.5204 - accuracy: 0.7500 - val_loss: 0.6673 - val_accuracy: 0.6719
Epoch 4/10
43/43 [==============================] - 5s 123ms/step - loss: 0.4709 - accuracy: 0.7602 - val_loss: 0.6917 - val_accuracy: 0.5938
Epoch 5/10
43/43 [==============================] - ETA: 0s - loss: 0.4378 - accuracy: 0.7851
Validation accuracy reached 80.0% or higher. Stopping training.
43/43 [==============================] - 5s 123ms/step - loss: 0.4378 - accuracy: 0.7851 - val_loss: 0.5685 - val_accuracy: 0.7500


In [230]:
# Save the model
model.save('/Users/damodargupta/Desktop/breast cancer prediction ann/model/breast_cancer_model_ann8.h5')
print("Model saved successfully.")

Model saved successfully.


# LOADING MODEL WHICH HAS THE HIGHEST ACCURACY AMONG ALL - 84.375 %

In [14]:
model = load_model('/Users/damodargupta/Desktop/breast cancer prediction ann/final-model.h5')


In [15]:
# Evaluate the model
_, accuracy = model.evaluate(test_generator, steps=validation_steps)
print(f"Accuracy: {accuracy}")

2023-07-12 17:24:46.193322: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2/2 [==============================] - 0s 28ms/step - loss: 0.3843 - accuracy: 0.8438
Accuracy: 0.84375


In [232]:
""